In [1]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd

In [22]:
stock = input("Enter stock ticker: ")

Enter stock ticker: aapl


In [23]:
apikey="EXjTead34W2WK09cgutAc4u10stA3u1R1J6w8sV4"

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

In [27]:
try:
    response = requests.get(url, headers=headers, params=querystring)
    response.raise_for_status()
    stock_json = response.json()

    if 'quoteResponse' in stock_json and len(stock_json['quoteResponse']['result']) > 0:
        
        # ticker and information about ticker
        stock_data = stock_json['quoteResponse']['result'][0]
        ticker = stock_data.get('symbol', 'N/A')
        print(f"Ticker name: {ticker}")
        full_name = stock_data.get('longName', 'N/A')
        print(f"Full Name of the Stock: {full_name}")
        current_price = stock_data.get('regularMarketPrice', 'N/A')
        print(f"Current Market Price: ${current_price}")
        
        finance_url = f"https://yfapi.net/v11/finance/quoteSummary/{ticker}"
        finance_response = requests.get(finance_url, headers=headers, params={"modules": "financialData"})
        finance_response.raise_for_status()
        financial_data = finance_response.json().get('quoteSummary', {}).get('result', [{}])[0].get('financialData', {})
        target_mean_price = financial_data.get('targetMeanPrice', {}).get('raw', 'N/A')
        print(f"Target Mean Price: ${target_mean_price}")
        
        week_high = stock_data.get('fiftyTwoWeekHigh', 'N/A')
        print(f"52 Week High: ${week_high}")
        week_low = stock_data.get('fiftyTwoWeekLow', 'N/A')
        print(f"52 Week Low: ${week_low}")
        
        # 5 current trending stocks
        trending_url = "https://yfapi.net/v1/finance/trending/US"
        trending_response = requests.get(trending_url, headers=headers)
        trending_response.raise_for_status()
        trending_json = trending_response.json()

        if 'finance' in trending_json and 'result' in trending_json['finance'] and trending_json['finance']['result']:
            trending_stocks = trending_json['finance']['result'][0].get('quotes', [])
            print("\n5 Current Trending Stocks:")
            trending_symbols = [stock['symbol'] for stock in trending_stocks[:5]]
            print(", ".join(trending_symbols))  
        else:
            print("Trending information unavailable")
        
        data = {"Ticker": [ticker], "Name": [full_name], "Current Price": [current_price], "Target Mean Price": [target_mean_price], "52 Week High": [week_high], "52 Week Low": [week_low]}
        
        df = pd.DataFrame(data)
        df.to_csv("stock.csv", index=False)

    else:
        print(f"No information available for '{stock}'")
 
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"An error occurred: {err}")


Ticker name: AAPL
Full Name of the Stock: Apple Inc.
Current Market Price: $227.79
Target Mean Price: $240.58
52 Week High: $237.23
52 Week Low: $164.08

5 Current Trending Stocks:
BTC-USD, XRP-USD, DOGE-USD, SCHD, NNN
